# Introduzione a Redis

## Nozioni Base
Redis è un in-memory data structure store open source (BSD licensed)usato come database, cache, message broker e streaming engine. Redis dispone di data structures come strings, hashes, lists, sets, sorted sets con diverse queries, bitmaps, hyperloglogs, geospatial indexes, e streams. Redis ha un built-in replica system che permette di creare repliche del medesimo DB, Lua scripting, LRU eviction, transazioni e diversi livelli di on-disk persistence.

Per ottenere il massimo delle performance, Redis utilizza un dataset in-memory. In base al caso d'uso, Redis può salvare i dati dumpando periodicamente il dataset su disco, oppure aggiungere ad un log ogni operazione (comando) effettuato.

Redis può essere usato su quasi tutti i linguaggi di programmazione disponibili.

## Chi usa Redis?
Le compagnie più note che usano Redis sono:
- Twitter
- Github
- Snapchat
- Cragslist
- StackOverflow

E molte altre ancora.

## Installazione
### Windows
Redis non è supportato su Windows, ma si può installare sfruttando la WSL2 (Windows Subsystem for Linux).
Con una shell WSL correttamente inizializzata si potranno seguire gli step di installazione su macchina Ubuntu/Debian per installare l'ultima versione di Redis.

```bash
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

sudo apt-get update
sudo apt-get install redis
```

E per avviare il servizio basterà lanciare il comando:
```bash
sudo service redis-server start
```

### Linux
Avendo in commercio una gran quantità di distribuzioni Linux, ogni package manager di riferimento ha una guida su come installare il pacchetto Redis.
Come spiegato nel capitolo per Windows, su sistemi Debian/Ubuntu possiamo aggiungere il Repository APT Ufficiale `packages.redis.io` e installare successivamente il pacchetto:
```bash
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

sudo apt-get update
sudo apt-get install redis
```

Alternativamente possiamo sfruttare il pacchetto Redis disponibile sullo Snapcraft Store e installarlo su tutte le distribuzioni che lo supportano:

```bash
sudo snap install redis
```

### MacOS
Per installare il pacchetto Redis su dispositivi macOS, sarà necessario in prima battuta installare Brew, un package manager creato per il Sistema Operativo di Cupertino.
Una volta installato correttamente si dovrà semplicemente lanciare il comando:
```bash
brew install redis
```

Per avviare il servizio si dovrà usare:
```bash
redis-server
```

Per fermare il servizio andrà premuto `CTRL-C`

## Pro e Contro
PRO:

- Velocità: Redis è estremamente veloce, in grado di gestire grandi quantità di dati in modo rapido e efficiente, grazie alla sua architettura in memoria.

- Scalabilità: Redis è in grado di scalare orizzontalmente, ovvero di aggiungere nodi al cluster per aumentare la capacità di gestione dei dati.

- API: Redis offre una vasta gamma di API per supportare la gestione dei dati, tra cui la memorizzazione di dati semplici, liste, set e mappe.

- Persistenza dei dati: Redis offre la possibilità di persistere i dati su disco, garantendo la durata dei dati in caso di malfunzionamenti del server.

- Architettura semplice: Redis è facile da utilizzare e da configurare, anche per gli utenti meno esperti.

CONTRO:

- Scalabilità: Anche se Redis è in grado di scalare orizzontalmente, aggiungendo nodi ai cluster, non sfrutta al massimo le capacità della singola macchina. Per questo sono state create versioni multi-thread ad esempio: [cachegrand.io](https://cachegrand.io/)

- Capacità limitata: Poiché Redis utilizza la memoria per archiviare i dati, la capacità di archiviazione è limitata dalla quantità di memoria disponibile sul server.

- Nessun supporto per SQL: Redis non supporta SQL, quindi non è adatto per applicazioni che richiedono complesse operazioni di query.

- Persistenza dei dati: Anche se Redis offre la possibilità di persistere i dati su disco, questa funzionalità può rallentare le prestazioni del server.

- Nessun supporto per transazioni complesse: Redis non supporta le transazioni complesse, il che lo rende meno adatto per le applicazioni che richiedono operazioni di transazione complesse e transazioni ACID.

## Librerie del DBMS
Quasi tutti i linguaggi di programmazione hanno disponibile un porting per connettersi al servizio Redis, lavorando con Python andremo nel dettaglio di quest'ultima.
### redis-py
**Installazione**

Per installare redis-py:
```bash
pip install redis-py
```

Per migliorare le performance si può utilizzare installa il pacchetto con il supporto ad `hiredis`, che offre un Response Parser compilato, che per molte situazioni richiede 0 modifiche.

```bash
pip install redis[hiredis]
```

Link alla documentazione di [redis-py](https://redis-py.readthedocs.io/en/stable/)

In [ ]:
import sys
!{sys.executable} -m pip install redis

## Creazione e Connessione al DB
Per facilitare lo sviluppo su più sistemi operativi, questo progetto dispone di un docker-compose.yml file, contenente il necessario per avviare un'istanza redis con due User Interfaces, quali `redis-cli` e `RedisInsight`.

Per avviare il servizio si può lanciare il comando:
```bash
make start
```

### Connessione al DB tramite redis-py

In [15]:
import redis

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

Prove comandi

## Operazioni CRUD
In Redis, le operazioni CRUD (Create, Read, Update, Delete) sono eseguite tramite comandi specifici per la manipolazione dei dati.

Ecco una panoramica delle operazioni CRUD in Redis:

- CREATE: L'operazione di creazione di un nuovo dato in Redis viene eseguita utilizzando il comando SET. Ad esempio, per creare un nuovo valore "valore1" associato alla chiave "chiave1", si può utilizzare il comando SET chiave1 valore1.

In [16]:
r.set('nome', 'Marco')

True

- READ: Per leggere i dati associati a una chiave, si può utilizzare il comando GET. Ad esempio, per leggere il valore associato alla chiave "chiave1", si può utilizzare il comando GET chiave1.

In [17]:
r.get('nome')

'Marco'

- UPDATE: L'operazione di aggiornamento di un dato esistente in Redis viene eseguita utilizzando il comando SET. Ad esempio, per aggiornare il valore associato alla chiave "chiave1" con il nuovo valore "valore2", si può utilizzare il comando SET chiave1 valore2.

In [ ]:
r.set('nome', 'Francesco')


- DELETE: Per eliminare una chiave e il valore associato, si può utilizzare il comando DEL. Ad esempio, per eliminare la chiave "chiave1" e il valore associato ad essa, si può utilizzare il comando DEL chiave1.

In [ ]:
r.del('nome')

Inoltre, Redis offre anche altri comandi per manipolare i dati in modo più specifico, come ad esempio il comando INCR per incrementare il valore numerico di una chiave, il comando LPUSH per aggiungere un valore a una lista, e il comando SADD per aggiungere un elemento a un set.

# Contributors
Lorenzo Emanuele Avallone 866163\
[inser here il tuo nominativo e matricola :D]